In [1]:
%pip install scipy torchsummary


[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib Pillow


[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip


In [3]:
%pip install opencv-python


[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
%pip install matplotlib


[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
%pip install pandas


[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [12]:
import os
import time
import scipy.io
import numpy as np
import torch
import torchvision
import matplotlib.pyplot as plt
from torchsummary import summary
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, Dataset
from torchvision.models import MobileNet_V3_Large_Weights
from torch.optim import Adam
import torch.optim as optim
from torchvision.models import mobilenet_v3_large, MobileNet_V3_Large_Weights
from torch import nn

## Load Dataset

**Define Custom Dataset**

In [7]:
### UNCOMMENT ONLY IF UNZIPPING DATASET ###

# import zipfile

# with zipfile.ZipFile("jpg.zip", "r") as zip_ref:
#     zip_ref.extractall("images")

In [8]:
# import os

# folder_path = "images/jpg"

# # Count all files (excluding subdirectories)
# file_count = sum(1 for file in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, file)))

# print(f"Number of files: {file_count}")


Number of files: 8189


In [7]:
from PIL import Image
from torch.utils.data import Dataset
import os
import torch
import scipy.io

class OxfordFlowersDataset(Dataset):
    def __init__(self, image_folder, label_file, transform=None):
        """
        :param image_folder: Path to the local folder containing images.
        :param label_file: Path to the .mat file containing labels.
        :param transform: Transformations to apply to the images.
        """
        self.image_folder = image_folder
        self.label_file = label_file
        self.transform = transform
        self.image_files = self._get_image_files()
        self.labels = self._load_labels()

    def _get_image_files(self):
        """Retrieve all image file names from the local folder."""
        image_files = [f for f in os.listdir(self.image_folder) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
        return sorted(image_files)  # Ensure images are loaded in order

    def _load_labels(self):
        """Load labels from a local .mat file."""
        label_data = scipy.io.loadmat(self.label_file)
        image_labels = label_data['labels'].flatten() - 1  # Convert 1-based to 0-based indexing
        return torch.tensor(image_labels, dtype=torch.long)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        file_name = self.image_files[idx]
        image_path = os.path.join(self.image_folder, file_name)
        label = self.labels[idx]

        try:
            # Open image using PIL
            image = Image.open(image_path).convert("RGB")

        except Exception as e:
            print(f"Error reading image {file_name}: {e}")
            return None, None

        # Apply transformations if specified
        if self.transform:
            image = self.transform(image)

        return image, label

## Training Prep

Proportion:
- 75:25 - train:val
- 128 batch size
- GPU enabled on RunPod
- weight decay
- dropout
- criterion for label smoothing
- learning rate scheduler (Cosine Annealing Learning Rate Scheduler)

Transformation:
- resize to 224x224
- horizonal flip
- rotate
- vertical flip
- affine
- perspective
- convert to tensor
- gaussian blur
- random erasing
- normalize image

In [24]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to match model input
    
    # Stronger Data Augmentation
    transforms.RandomRotation(degrees=40),  # Increased rotation range
    transforms.RandomHorizontalFlip(p=0.5),  # Flip horizontally
    transforms.RandomVerticalFlip(p=0.2),  # Flip vertically
    transforms.RandomAffine(degrees=0, translate=(0.2, 0.2)),  # Increased translation
    transforms.RandomPerspective(distortion_scale=0.3, p=0.5),  # Higher perspective change
    transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.3, hue=0.2),  # Stronger color jitter
    transforms.GaussianBlur(kernel_size=3, sigma=(0.1, 2.0)),  # Adding blur

    # Convert to Tensor & Normalize
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Standard normalization
])

dataset = OxfordFlowersDataset(
    image_folder = "images/jpg", 
    label_file = "imagelabels.mat",
    transform   = transform
)

# Split dataset into training and validation
train_size = int(0.8 * len(dataset))  # 80% train, 20% validation
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

# Data Loaders
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=4, pin_memory=True, prefetch_factor=2)
val_loader = DataLoader(val_dataset, batch_size=128, shuffle=False, num_workers=4, pin_memory=True, prefetch_factor=2)



In [9]:
print(f"Number of images: {len(dataset.image_files)}")
print(f"Number of labels: {len(dataset.labels)}")

Number of images: 8189
Number of labels: 8189


In [25]:
# Load MobileNetV3 model
model = mobilenet_v3_large(weights=MobileNet_V3_Large_Weights.IMAGENET1K_V1)

# Modify classifier (Add Dropout before Final Layer)
model.classifier[2] = nn.Dropout(p=0.4)  # Added dropout to reduce overfitting
model.classifier[3] = nn.Linear(model.classifier[3].in_features, 102)  # Adjust for 102 classes

# Define device
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# Check if the model is on GPU
for param in model.parameters():
    assert param.device == device, f"Model parameter is on {param.device}, not {device}!"

print("Model successfully moved to:", device)

# Optimizer with Weight Decay for Regularization
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)

# Loss Function with Label Smoothing
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)

# Learning Rate Scheduler (Reduce LR on Plateau)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3)

Model successfully moved to: cuda:0


In [17]:
# criterion = nn.CrossEntropyLoss()
# optimizer = Adam(model.parameters(), lr=0.001)

## Model Training

In [26]:
def train_model(model, criterion, optimizer, train_loader, val_loader, num_epochs=10):
    """Train the model and return losses, accuracies, and elapsed time."""
    train_losses, val_losses = [], []
    train_accuracies, val_accuracies = [], []

    start_time = time.time()

    for epoch in range(num_epochs):
        # Training phase
        model.train()
        train_loss, correct, total = 0.0, 0, 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

        train_acc = 100. * correct / total
        train_losses.append(train_loss / len(train_loader))
        train_accuracies.append(train_acc)

        # Validation phase
        model.eval()
        val_loss, correct, total = 0.0, 0, 0

        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)

                outputs = model(images)
                loss = criterion(outputs, labels)

                val_loss += loss.item()
                _, predicted = outputs.max(1)
                total += labels.size(0)
                correct += predicted.eq(labels).sum().item()

        val_acc = 100. * correct / total
        val_losses.append(val_loss / len(val_loader))
        val_accuracies.append(val_acc)

        # Adjust learning rate if validation loss stagnates
        scheduler.step(val_loss / len(val_loader))
        
        print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_losses[-1]:.4f}, "
              f"Train Acc: {train_accuracies[-1]:.2f}%, Val Loss: {val_losses[-1]:.4f}, "
              f"Val Acc: {val_accuracies[-1]:.2f}%")

    end_time = time.time()
    elapsed_time = end_time - start_time

    return model, train_losses, val_losses, train_accuracies, val_accuracies, elapsed_time

In [23]:
# def plot_and_save_curves(train_losses, val_losses, train_accuracies, val_accuracies):
#     """Plot loss and accuracy curves and save them as images."""
    
#     # Plot losses
#     plt.figure()
#     plt.plot(train_losses, label='Training Loss', marker='o')
#     plt.plot(val_losses, label='Validation Loss', marker='s')
#     plt.xlabel('Epochs')
#     plt.ylabel('Loss')
#     plt.legend()
#     plt.title('Learning Curve: Loss')
#     plt.savefig("loss_curve.png")  # Save loss curve
#     plt.close()

#     # Plot accuracies
#     plt.figure()
#     plt.plot(train_accuracies, label='Training Accuracy', marker='o')
#     plt.plot(val_accuracies, label='Validation Accuracy', marker='s')
#     plt.xlabel('Epochs')
#     plt.ylabel('Accuracy (%)')
#     plt.legend()
#     plt.title('Learning Curve: Accuracy')
#     plt.savefig("accuracy_curve.png")  # Save accuracy curve
#     plt.close()

#     print("Saved plots as 'loss_curve.png' and 'accuracy_curve.png'")

In [27]:
# Train the model
model, train_losses, val_losses, train_accuracies, val_accuracies, elapsed_time = train_model(
    model, criterion, optimizer, train_loader, val_loader, num_epochs=100
)

print(f"\nElapsed time: {elapsed_time:.2f} seconds\n")

# Plot and save learning curves
plot_and_save_curves(train_losses, val_losses, train_accuracies, val_accuracies)

Epoch 1/100, Train Loss: 2.5392, Train Acc: 51.61%, Val Loss: 3.9644, Val Acc: 19.72%
Epoch 2/100, Train Loss: 1.3844, Train Acc: 85.19%, Val Loss: 2.1858, Val Acc: 59.95%
Epoch 3/100, Train Loss: 1.2027, Train Acc: 91.01%, Val Loss: 1.5201, Val Acc: 82.05%
Epoch 4/100, Train Loss: 1.1140, Train Acc: 93.54%, Val Loss: 1.3009, Val Acc: 86.57%
Epoch 5/100, Train Loss: 1.0575, Train Acc: 95.28%, Val Loss: 1.2415, Val Acc: 88.71%
Epoch 6/100, Train Loss: 1.0285, Train Acc: 95.88%, Val Loss: 1.2788, Val Acc: 87.42%
Epoch 7/100, Train Loss: 0.9895, Train Acc: 97.36%, Val Loss: 1.2233, Val Acc: 88.64%
Epoch 8/100, Train Loss: 0.9616, Train Acc: 97.89%, Val Loss: 1.1925, Val Acc: 89.19%
Epoch 9/100, Train Loss: 0.9530, Train Acc: 97.85%, Val Loss: 1.1299, Val Acc: 92.43%
Epoch 10/100, Train Loss: 0.9409, Train Acc: 98.14%, Val Loss: 1.2208, Val Acc: 88.10%
Epoch 11/100, Train Loss: 0.9443, Train Acc: 98.12%, Val Loss: 1.0885, Val Acc: 92.61%
Epoch 12/100, Train Loss: 0.9418, Train Acc: 98.03%,

NameError: name 'plot_and_save_curves' is not defined

In [28]:
torch.save(model.state_dict(), 'model.pth')

In [29]:
%pip install plotly


[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def plot_interactive_curves(train_losses, val_losses, train_accuracies, val_accuracies):
    """Plot interactive loss and accuracy curves using Plotly."""
    
    epochs = list(range(1, len(train_losses) + 1))  # X-axis: Epochs

    # Create subplot: Loss (row 1) and Accuracy (row 2)
    fig = make_subplots(rows=2, cols=1, 
                        shared_xaxes=True, 
                        subplot_titles=("Loss Curve", "Accuracy Curve"))

    # Plot Losses
    fig.add_trace(go.Scatter(x=epochs, y=train_losses, mode='lines+markers',
                             name='Training Loss', marker=dict(symbol="circle")),
                  row=1, col=1)

    fig.add_trace(go.Scatter(x=epochs, y=val_losses, mode='lines+markers',
                             name='Validation Loss', marker=dict(symbol="square")),
                  row=1, col=1)

    # Plot Accuracies
    fig.add_trace(go.Scatter(x=epochs, y=train_accuracies, mode='lines+markers',
                             name='Training Accuracy', marker=dict(symbol="circle")),
                  row=2, col=1)

    fig.add_trace(go.Scatter(x=epochs, y=val_accuracies, mode='lines+markers',
                             name='Validation Accuracy', marker=dict(symbol="square")),
                  row=2, col=1)

    # Update layout
    fig.update_layout(
        height=900,  # Increase height
        width=1200,  # Increase width
        title_text="Training Progress",
        xaxis2=dict(title="Epochs"),
        yaxis=dict(title="Loss"),
        yaxis2=dict(title="Accuracy (%)"),
    )

    # Save plot as image (PNG)
    fig.write_image("training_progress.png")
    
    # Show interactive plot
    fig.show()

plot_interactive_curves(train_losses, val_losses, train_accuracies, val_accuracies)


NameError: name 'train_losses' is not defined

In [ ]:

# Extract best accuracy and corresponding epoch
best_train_acc = max(train_accuracies)
best_train_acc_epoch = np.argmax(train_accuracies) + 1

best_val_acc = max(val_accuracies)
best_val_acc_epoch = np.argmax(val_accuracies) + 1

# Extract best (lowest) loss and corresponding epoch
best_train_loss = min(train_losses)
best_train_loss_epoch = np.argmin(train_losses) + 1

best_val_loss = min(val_losses)
best_val_loss_epoch = np.argmin(val_losses) + 1

# Create a summary dictionary
best_metrics_summary = {
    "Metric": ["Best Training Accuracy", "Best Validation Accuracy", "Best Training Loss", "Best Validation Loss"],
    "Value": [f"{best_train_acc:.2f}%", f"{best_val_acc:.2f}%", f"{best_train_loss:.4f}", f"{best_val_loss:.4f}"],
    "Epoch": [best_train_acc_epoch, best_val_acc_epoch, best_train_loss_epoch, best_val_loss_epoch]
}

# Convert to dataframe and display
import pandas as pd
best_metrics_df = pd.DataFrame(best_metrics_summary)

best_metrics_df